In [18]:
# Importing required libraries.
import zipfile
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [19]:
# Loading the MNIST dataset from the local folder.
def extract_zip(zip_path, extract_to='Data'):
    if not os.path.exists(extract_to):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    return extract_to

def load_data(root_dir):
    images, labels = [], []
    for root, dirs, files in os.walk(root_dir):
        if os.path.basename(root).isdigit():
            label = int(os.path.basename(root))
            for img_file in os.listdir(root):
                img_path = os.path.join(root, img_file)
                if os.path.isfile(img_path):
                    img = Image.open(img_path).convert('L')
                    image = np.array(img)
                    images.append(image)
                    labels.append(label)
    return np.array(images), np.array(labels)

In [23]:
# Central code employing HOG Feature extraction and using classifier models on it.
if __name__ == "__main__":
    zip_path = "./Data.zip"
    extracted_dir = extract_zip(zip_path)
    training_dir = os.path.join(extracted_dir, "MNIST Dataset JPG format", "MNIST - JPG - training")
    images, labels = load_data(training_dir)

    # Reduced HOG Parameter Grid
    hog_params_grid = [
        {'cell_size': 7, 'block_size': 14, 'block_stride': 7, 'nbins': 9},
        {'cell_size': 8, 'block_size': 16, 'block_stride': 4, 'nbins': 9},
        {'cell_size': 7, 'block_size': 21, 'block_stride': 7, 'nbins': 9}
    ]

    # Faster Classifiers with Simplified Grids
    classifiers = [
        ('SVM', SVC(), {'C': [1, 10], 'gamma': ['scale']}),  # Reduced options
        ('Logistic Regression', LogisticRegression(max_iter=1000), {'C': [0.1, 1]}),
        ('LDA', LinearDiscriminantAnalysis(), {'solver': ['svd']}),
    ]

    best_overall = {'accuracy': 0}

    for hog_params in hog_params_grid:
        print(f"\n=== Testing HOG Params: {hog_params} ===")

        # HOG initialization (same as before)
        cell_size = (hog_params['cell_size'], hog_params['cell_size'])
        block_size = (hog_params['block_size'], hog_params['block_size'])
        block_stride = (hog_params['block_stride'], hog_params['block_stride'])
        nbins = hog_params['nbins']

        hog = cv2.HOGDescriptor(
            (28, 28), block_size, block_stride, cell_size, nbins
        )

        # Compute HOG features (optimized)
        X = np.array([hog.compute(img).flatten() for img in images])
        y = labels

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fast classifier tuning
        for clf_name, clf, param_grid in classifiers:
            grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1)
            grid.fit(X_train, y_train)
            best_acc = accuracy_score(y_test, grid.best_estimator_.predict(X_test))

            print(f"{clf_name}: {best_acc:.4f} (Params: {grid.best_params_})")

            if best_acc > best_overall['accuracy']:
                best_overall = {
                    'accuracy': best_acc,
                    'hog_params': hog_params,
                    'clf_name': clf_name,
                    'clf_params': grid.best_params_
                }

    print("\n=== Final Best ===")
    print(f"Accuracy: {best_overall['accuracy']:.4f}")
    print(f"HOG Config: {best_overall['hog_params']}")
    print(f"Best Model: {best_overall['clf_name']}")
    print(f"Model Params: {best_overall['clf_params']}")


=== Testing HOG Params: {'cell_size': 7, 'block_size': 14, 'block_stride': 7, 'nbins': 9} ===
SVM: 0.9905 (Params: {'C': 10, 'gamma': 'scale'})
Logistic Regression: 0.9822 (Params: {'C': 1})
LDA: 0.9735 (Params: {'solver': 'svd'})

=== Testing HOG Params: {'cell_size': 8, 'block_size': 16, 'block_stride': 4, 'nbins': 9} ===
SVM: 0.9902 (Params: {'C': 10, 'gamma': 'scale'})
Logistic Regression: 0.9815 (Params: {'C': 1})
LDA: 0.9744 (Params: {'solver': 'svd'})

=== Testing HOG Params: {'cell_size': 7, 'block_size': 21, 'block_stride': 7, 'nbins': 9} ===
SVM: 0.9904 (Params: {'C': 10, 'gamma': 'scale'})
Logistic Regression: 0.9792 (Params: {'C': 1})
LDA: 0.9711 (Params: {'solver': 'svd'})

=== Final Best ===
Accuracy: 0.9905
HOG Config: {'cell_size': 7, 'block_size': 14, 'block_stride': 7, 'nbins': 9}
Best Model: SVM
Model Params: {'C': 10, 'gamma': 'scale'}
